In [1]:
import pandas as pd
import numpy as np
import requests
import urllib
import os

In [2]:
# Read in files
filenames = {'obs': 'observations.csv', 'loc': 'locations.csv', 'nme': 'name_classifications.csv'}
data = dict()
for file in filenames:
    data[file] = pd.read_csv(os.path.join('..', 'data', 'input', filenames[file]), sep='\t')

In [3]:
# Clean observations dataframe (relevant columns, drop na)
data['obs'] = data['obs'][['id', 'name_id', 'when', 'location_id']]
data['obs'].dropna(inplace=True, ignore_index=True)
data['obs'].drop_duplicates(inplace=True, ignore_index=True)
data['obs'].rename(columns={'id': 'obs_id'}, inplace=True)
print(data['obs'].shape)
data['obs'].head()

(498500, 4)


,obs_id,name_id,when,location_id
0,1,2,2004-07-13,214
1,2,3,2004-07-17,53
2,3,4,2002-01-08,60
3,4,4,1996-01-15,5
4,5,5,2002-12-28,36


In [4]:
# Clean locations file
data['loc'] = data['loc'][['id', 'name', 'north', 'west']]
data['loc'].dropna(inplace=True, ignore_index=True)
data['loc'].drop_duplicates(inplace=True, ignore_index=True)
data['loc'].rename(columns={'id': 'location_id'}, inplace=True)
data['loc'].head()

,location_id,name,north,west
0,1,"Albion, Mendocino Co., California, USA",39.2408,-123.778
1,2,"Burbank, Los Angeles Co., California, USA",34.2217,-118.370
2,3,"Mitrula Marsh, Tahoe National Forest, Sierra C...",39.6272,-120.620
3,4,"Salt Point State Park, Sonoma Co., California,...",38.5923,-123.343
4,5,"Gualala, Mendocino Co., California, USA",38.7868,-123.557


In [5]:
# Add is_sc column to locations dataframe
data['loc']['name'] = data['loc']['name'].str.lower()
data['loc']['split_name'] = data['loc']['name'].str.replace(' ', '').str.split(',')
data['loc']['is_sc'] = data['loc']['name'].str.contains('santa cruz', case=False)
data['loc']['is_sc'] = data['loc'].apply(lambda x: (x['split_name'][-1] == 'usa' and x['split_name'][-2] == 'california') if x['is_sc'] else False, axis=1)
print(data['loc'].shape)
data['loc'].head()

(26027, 6)


,location_id,name,north,west,split_name,is_sc
0,1,"albion, mendocino co., california, usa",39.2408,-123.778,"[albion, mendocinoco., california, usa]",False
1,2,"burbank, los angeles co., california, usa",34.2217,-118.370,"[burbank, losangelesco., california, usa]",False
2,3,"mitrula marsh, tahoe national forest, sierra c...",39.6272,-120.620,"[mitrulamarsh, tahoenationalforest, sierraco.,...",False
3,4,"salt point state park, sonoma co., california,...",38.5923,-123.343,"[saltpointstatepark, sonomaco., california, usa]",False
4,5,"gualala, mendocino co., california, usa",38.7868,-123.557,"[gualala, mendocinoco., california, usa]",False


In [6]:
# Clean name dataframe
data['nme'] = data['nme'][['name_id', 'domain', 'kingdom', 'phylum', 'class', 'order', 'family']]
data['nme'].dropna(inplace=True, ignore_index=True)
data['nme'].drop_duplicates(inplace=True, ignore_index=True)
data['nme'].head()

,name_id,domain,kingdom,phylum,class,order,family
0,2,Eukarya,Fungi,Ascomycota,Sordariomycetes,Xylariales,Xylariaceae
1,3,Eukarya,Fungi,Ascomycota,Sordariomycetes,Xylariales,Xylariaceae
2,4,Eukarya,Fungi,Ascomycota,Sordariomycetes,Xylariales,Xylariaceae
3,5,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae
4,7,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Boletales,Boletaceae


In [7]:
# Define custom corrections for name data
data['nme']['class'] = np.where(data['nme']['order'].eq('Physarales'), 'Myxogastria', data['nme']['class'])
data['nme']['phylum'] = np.where(data['nme']['class'].eq('Myxogastria'), 'Mycetozoa', data['nme']['phylum'])
data['nme']['kingdom'] = np.where(data['nme']['phylum'].eq('Mycetozoa'), 'Amoebozoa', data['nme']['kingdom'])

In [8]:
# Get location data for sc only
sc_loc_df = data['loc'].query('is_sc == True')[['location_id']]
sc_loc_df.head()

,location_id
53,58
54,60
56,62
63,70
64,72


In [9]:
# Add observations to santa cruz locaitons, drop cases with no observations
all_df = sc_loc_df.merge(data['obs'], how='left', on='location_id')
all_df.dropna(subset=['obs_id'], inplace=True, ignore_index=True)
print(all_df.shape)
all_df.head()

(8149, 4)


,location_id,obs_id,name_id,when
0,58,6.0,61758.0,2002-01-08
1,58,7.0,6.0,2005-01-07
2,58,13.0,25700.0,2005-01-07
3,58,18.0,554.0,2005-01-09
4,58,19.0,1016.0,2005-01-07


In [10]:
# Add name data 
all_df = all_df.merge(data['nme'], how='left', on='name_id')
all_df.dropna(subset=['domain', 'kingdom', 'phylum', 'class', 'order', 'family'], inplace=True, ignore_index=True)
print(all_df.shape)
all_df.head(2)

(6776, 10)


,location_id,obs_id,name_id,when,domain,kingdom,phylum,class,order,family
0,58,6.0,61758.0,2002-01-08,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae
1,58,13.0,25700.0,2005-01-07,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Pluteaceae


In [11]:
# Add yead and month data and keep only rows where year >= 1994
all_df['when'] = pd.to_datetime(all_df['when'])
all_df['year'] = all_df['when'].dt.year
all_df['month'] = all_df['when'].dt.month
all_df = all_df.query('year >= 1994')
all_df.head(2)

,location_id,obs_id,name_id,when,domain,kingdom,phylum,class,order,family,year,month
0,58,6.0,61758.0,2002-01-08,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,2002,1
1,58,13.0,25700.0,2005-01-07,Eukarya,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Pluteaceae,2005,1


In [12]:
# Constrct df with all desired dates
dates_df = pd.DataFrame([(y, m) for y in range(1994, 2024) for m in range(1, 13)], columns=['year', 'month'])
dates_df.head()

,year,month
0,1994,1
1,1994,2
2,1994,3
3,1994,4
4,1994,5


In [13]:
# Get list of all phylum-order combinations to create
po_df = all_df[['phylum', 'order']].drop_duplicates(ignore_index=True)
po_df.head()

,phylum,order
0,Basidiomycota,Agaricales
1,Basidiomycota,Boletales
2,Basidiomycota,Polyporales
3,Ascomycota,Pezizales
4,Basidiomycota,Thelephorales


In [14]:
# Create datagrame of all date/phylum/order combinations to consider
po_df['tmp'] = 1
dates_df['tmp'] = 1
combo_df = po_df.merge(dates_df, on='tmp')
combo_df.drop(columns=['tmp'], inplace=True)
combo_df.head()

,phylum,order,year,month
0,Basidiomycota,Agaricales,1994,1
1,Basidiomycota,Agaricales,1994,2
2,Basidiomycota,Agaricales,1994,3
3,Basidiomycota,Agaricales,1994,4
4,Basidiomycota,Agaricales,1994,5


In [15]:
# Count number of observations by phylum, order, year, and month
count_df = all_df.groupby(by=['phylum', 'order', 'year', 'month'])['obs_id'].count().reset_index()
count_df.rename(columns={'obs_id': 'total_obs'}, inplace=True)
count_df.head()

,phylum,order,year,month,total_obs
0,Ascomycota,Amphisphaeriales,2014,5,1
1,Ascomycota,Arthoniales,2013,3,1
2,Ascomycota,Arthoniales,2013,4,1
3,Ascomycota,Arthoniales,2017,5,2
4,Ascomycota,Caliciales,2015,1,1


In [16]:
# Join desired combinations of what/when with actual observation counts and fillna with 0
count_df = combo_df.merge(count_df, how='left', on=['phylum', 'year', 'month', 'order'])
count_df['total_obs'] = count_df['total_obs'].fillna(0)
count_df.head()

,phylum,order,year,month,total_obs
0,Basidiomycota,Agaricales,1994,1,4.0
1,Basidiomycota,Agaricales,1994,2,1.0
2,Basidiomycota,Agaricales,1994,3,0.0
3,Basidiomycota,Agaricales,1994,4,0.0
4,Basidiomycota,Agaricales,1994,5,0.0


In [17]:
# Find phylums with highest observation counts
count_df.groupby(by=['phylum'])['total_obs'].sum()

phylum
Ascomycota        502.0
Basidiomycota    6127.0
Mucoromycota        7.0
Mycetozoa           9.0
Myxomycota        118.0
Zoopagomycota       1.0
Zygomycota          2.0
Name: total_obs, dtype: float64

In [18]:
# Replace Myxomycota with Mycetozoa as technically Myxomycota is not a phylum
count_df['phylum'] = np.where(count_df['phylum'].eq('Myxomycota'), 'Mycetozoa', count_df['phylum'])

In [19]:
# Find phylums with highest observation counts
count_df.groupby(by=['phylum'])['total_obs'].sum()

phylum
Ascomycota        502.0
Basidiomycota    6127.0
Mucoromycota        7.0
Mycetozoa         127.0
Zoopagomycota       1.0
Zygomycota          2.0
Name: total_obs, dtype: float64

In [20]:
# Replace phylums with smaller observation counts with 'other'
main_phylums = ['Mycetozoa', 'Basidiomycota', 'Ascomycota']
count_df['phylum'] = np.where(~count_df['phylum'].isin(main_phylums), 'Other', count_df['phylum'])
count_df.head()

,phylum,order,year,month,total_obs
0,Basidiomycota,Agaricales,1994,1,4.0
1,Basidiomycota,Agaricales,1994,2,1.0
2,Basidiomycota,Agaricales,1994,3,0.0
3,Basidiomycota,Agaricales,1994,4,0.0
4,Basidiomycota,Agaricales,1994,5,0.0


In [22]:
# See count per phylum
count_df.groupby(by=['phylum'])['total_obs'].sum()

phylum
Ascomycota        502.0
Basidiomycota    6127.0
Mycetozoa         127.0
Other              10.0
Name: total_obs, dtype: float64

In [25]:
# This is our main count dataframe
count_df.head()

,phylum,order,year,month,total_obs
0,Basidiomycota,Agaricales,1994,1,4.0
1,Basidiomycota,Agaricales,1994,2,1.0
2,Basidiomycota,Agaricales,1994,3,0.0
3,Basidiomycota,Agaricales,1994,4,0.0
4,Basidiomycota,Agaricales,1994,5,0.0
